### Explicação do Código

    Extração de Áudio com FFmpeg:
        O FFmpeg extrai o áudio do vídeo e converte para um arquivo .wav mono com taxa de amostragem de 16 kHz, compatível com o Whisper.

    Carregamento do Modelo Whisper:
        O modelo Whisper e seu tokenizer são carregados do Hugging Face. Certifique-se de estar usando o dispositivo correto (CPU ou GPU).

    Transcrição do Áudio:
        A função pipeline facilita a transcrição do áudio. O texto resultante da transcrição é extraído e exibido.

#Pré-requisitos

    Certifique-se de ter instalado as bibliotecas necessárias:

In [ ]:
!pip install transformers ffmpeg-python torch


### Tenha um arquivo de vídeo no formato suportado pela Biblioteca FFmpeg.

In [ ]:
import ffmpeg
from transformers import pipeline, WhisperProcessor, WhisperForConditionalGeneration
import torch

# Função para extrair áudio de um arquivo de vídeo
def extract_audio(video_path, audio_path):
    """
    Extrai o áudio de um arquivo de vídeo e o salva como arquivo .wav.
    """
    try:
        (
            ffmpeg
            .input(video_path)
            .output(audio_path, format="wav", ac=1, ar="16000")  # Mono (ac=1), taxa de amostragem 16kHz (ar=16000)
            .overwrite_output()
            .run()
        )
        print(f"Áudio extraído com sucesso para {audio_path}")
    except Exception as e:
        print(f"Erro ao extrair áudio: {e}")

# Caminho do arquivo de vídeo
video_file = "seu_video.mp4"  # Substitua pelo caminho do seu arquivo de vídeo
audio_file = "extraido_audio.wav"  # Arquivo de saída do áudio

# Extrair o áudio
extract_audio(video_file, audio_file)

# Carregar o modelo Whisper da Hugging Face
print("Carregando modelo Whisper...")
device = "cuda" if torch.cuda.is_available() else "cpu"
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base").to(device)
processor = WhisperProcessor.from_pretrained("openai/whisper-base")

# Função para transcrever áudio
def transcribe_audio(audio_path):
    """
    Transcreve o áudio usando o modelo Whisper da Hugging Face.
    """
    # Carregar o pipeline de transcrição
    transcriber = pipeline(
        "automatic-speech-recognition",
        model=model,
        tokenizer=processor.tokenizer,
        device=0 if torch.cuda.is_available() else -1
    )

    # Transcrever o áudio
    print("Transcrevendo áudio...")
    try:
        result = transcriber(audio_path)
        print("Transcrição concluída!")
        return result["text"]
    except Exception as e:
        print(f"Erro na transcrição: {e}")
        return None

# Transcrever o áudio extraído
transcription = transcribe_audio(audio_file)

# Exibir a transcrição
if transcription:
    print("Transcrição do áudio:")
    print(transcription)
else:
    print("Não foi possível transcrever o áudio.")